# Why they are used?

### 1. Web
        - Web Content Management Systems and small web apps use them to save credentials (DB or Mail user and passwords)
        - App Configurations (web.xml)
### 2. Scientific Softwares
        - Saving the path to libraries or files
        - Configurations for different experiments
### 3. Local Softwares
        - Games (the herding env)
        - Mail Clients
        - ...
### Alternative:
        - They can be hard coded which is an inexpensive and more readble way for small apps or apps with limited use.
        - Using command line arguments. This is great for shell scripts. All the options can be generated by an external program and fed to the main app.

# In Python

Here we cover all the python libraries for this.

1. Hard coding
2. INI files
3. YAML files
4. JSON files
5. XML files
6. Command line (argparse library)

In [1]:
# INSTALLS
!pip -q install configparser bs4 lxml

You are using pip version 10.0.1, however version 20.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


# 1- Hard Coding
can be done by python dictionaries, a class or python library:

In [2]:
class Agent:
    def __init__(self, config):
        print("Setting up the agent with these\n-------------")
        print("Layers:"+ str(config['layer_nodes']))
        print("activations:"+ str(config['activations']))
        print("loss:"+ str(config['loss']))
        print("learning Rate:"+ str(float(config['learning_rate'])))
        
agent_conf = {
    "layer_nodes":[64,64,4],
    "activations":["relu", "relu", ""],
    "loss":"mse",
    "optimizer":"adam",
    "learning_rate":5e-4
}

agent = Agent(agent_conf)

Setting up the agent with these
-------------
Layers:[64, 64, 4]
activations:['relu', 'relu', '']
loss:mse
learning Rate:0.0005


# 2- INI Files
are mainly used in windows apps. Thus, the Windows operating system and most softwares develops by and for Windows use this standard. It looks like this:

```bash
[section1]
option1=value1
option2=value2

[section2]
option3=value3
option4=value4
```
In python, the **ConfigParser** package can process these files.

In [3]:
%%file experiment.ini
[agent]
layer_nodes=64,64,4
activations="relu", "relu", ""
loss:mse
optimizer:adam
learning_rate:5e-4

[env]
agents=4
fog=true

Writing experiment.ini


In [4]:
from configparser import ConfigParser

configs = ConfigParser()

# Opening the file
with open('experiment.ini', 'r') as f:
    configs.read_file(f)

# Travesal
for section in configs.sections():
    print("Processing Section: "+section+"\n-------")
    for option in configs.options(section):
        print("Processing "+ option + " -> " + configs.get(section,option))
    print("\n")

layers = configs.get("agent","layer_nodes").split(",")
print("Processing list of values: "+ str(layers))

no_agents = configs.get("env","agents")
print("Number of agents: " + str(no_agents) + ". Its type is " + str(type(no_agents)) +\
      ". Don't forget to cast it before using it.")

Processing Section: agent
-------
Processing layer_nodes -> 64,64,4
Processing activations -> "relu", "relu", ""
Processing loss -> mse
Processing optimizer -> adam
Processing learning_rate -> 5e-4


Processing Section: env
-------
Processing agents -> 4
Processing fog -> true


Processing list of values: ['64', '64', '4']
Number of agents: 4. Its type is <class 'str'>. Don't forget to cast it before using it.


In [5]:
# Cleaning up
!rm experiment.ini

# 2- YAML
YAML (rhymes with camel) is a configuration file popular among programmers especially python developers. As wikipedia put it:
> YAML (a recursive acronym for "YAML Ain't Markup Language") is a human-readable data-serialization language. It is commonly used for configuration files and in applications where data is being stored or transmitted. YAML targets many of the same communications applications as Extensible Markup Language (XML) but has a minimal syntax which intentionally differs from SGML. It uses both Python-style indentation to indicate nesting, and a more compact format that uses [...] for lists and {...} for maps making YAML 1.2 a superset of JSON. The official recommended filename extension for YAML files has been *.yaml* since 2006. [But .yml is more popular]

YAML is JSON's official subset.

In [6]:
%%file experiment.yml
agent:
    algorithm: ppo
    network:
        - layer_nodes=64,64,4
        - activations="relu", "relu", ""
        - loss:mse
        - optimizer:adam
        - learning_rate:5e-4

env:
    agents: 4
    fog: true

Writing experiment.yml


In [7]:
import yaml

with open("experiment.yml", "r") as ymlfile:
    configs = yaml.load(ymlfile)

print("Sections are:\n---------")
for section in configs:
    print(section)
print("\n")

print("Each section is a dictionary:\n---------")
print('configs["agent"] = ' + str(configs["agent"]))
print()
print('configs["env"] = ' + str(configs["env"]))
print()

print("Here's the network config which comes as a python array: " + str(configs["agent"]["network"]))
nodes = configs["agent"]["network"][0].split("=")[1].split(",")
nodes

Sections are:
---------
agent
env


Each section is a dictionary:
---------
configs["agent"] = {'algorithm': 'ppo', 'network': ['layer_nodes=64,64,4', 'activations="relu", "relu", ""', 'loss:mse', 'optimizer:adam', 'learning_rate:5e-4']}

configs["env"] = {'agents': 4, 'fog': True}

Here's the network config which comes as a python array: ['layer_nodes=64,64,4', 'activations="relu", "relu", ""', 'loss:mse', 'optimizer:adam', 'learning_rate:5e-4']


['64', '64', '4']

In [8]:
!rm experiment.yml

# 3- JSON
is short of Javascript Object Notation. It's mainly used for data communication (web services) and storage (MongoDB). Due to its wide-spread support, it's also a good option for a configuration file.

In [9]:
%%file experiment.json
{
    "agent": {
    "algorithm": "ppo",
    "network":{
        "layer_nodes":["64","64","4"],
        "activations":["relu", "relu",""],
        "loss":"mse",
        "optimizer":"adam",
        "learning_rate":"5e-4"
        }
    },

    "env": {
        "agents": "4",
        "fog": "true"
    }
}

Writing experiment.json


In [10]:
import json, pprint

with open("experiment.json", "r") as json_config:
    configs = json.load(json_config)

pprint.pprint(configs)

print(configs["agent"]["network"]["layer_nodes"])

{'agent': {'algorithm': 'ppo',
           'network': {'activations': ['relu', 'relu', ''],
                       'layer_nodes': ['64', '64', '4'],
                       'learning_rate': '5e-4',
                       'loss': 'mse',
                       'optimizer': 'adam'}},
 'env': {'agents': '4', 'fog': 'true'}}
['64', '64', '4']


In [11]:
!rm experiment.json

# 4. XML

In [12]:
%%file experiment.xml
<config>
    <agent>
    <algorithm>ppo</algorithm>
        <network>
            <layer_nodes>
                <li>64</li>
                <li>64</li>
                <li>4</li>
            </layer_nodes>
            <activations>
                <li>relu</li>
                <li>relu</li>
                <li></li>
            </activations>
            <loss>mse</loss>
            <optimizer>adam</optimizer>
            <learning_rate>5e-4</learning_rate>
        </network>
    </agent>

    <env>
        <agents>4</agents>
        <fog>true</fog>
    </env>
</config>

Writing experiment.xml


In [13]:
from bs4 import BeautifulSoup
import lxml

with open("experiment.xml","r") as xml_file:
    contents = xml_file.read()

configs = BeautifulSoup(contents, "lxml")

print(configs.agent.algorithm.contents)
print(configs.agent.network.layer_nodes.contents[1].contents)
print(configs.env.fog.contents)

configs

['ppo']
['64']
['true']


<html><body><config>
<agent>
<algorithm>ppo</algorithm>
<network>
<layer_nodes>
<li>64</li>
<li>64</li>
<li>4</li>
</layer_nodes>
<activations>
<li>relu</li>
<li>relu</li>
<li></li>
</activations>
<loss>mse</loss>
<optimizer>adam</optimizer>
<learning_rate>5e-4</learning_rate>
</network>
</agent>
<env>
<agents>4</agents>
<fog>true</fog>
</env>
</config></body></html>

In [14]:
!rm experiment.xml

# Command Line

The argparse module makes it easy to write user-friendly command-line interfaces. The program defines what arguments it requires, and argparse will figure out how to parse those out of sys.argv. The argparse module also automatically generates help and usage messages and issues errors when users give the program invalid arguments.

```python
import argparse

parser = argparse.ArgumentParser(description='Process some integers.')
parser.add_argument('integers', metavar='N', type=int, nargs='+',
                    help='an integer for the accumulator')
parser.add_argument('--sum', dest='accumulate', action='store_const',
                    const=sum, default=max,
                    help='sum the integers (default: find the max)')

args = parser.parse_args()
print(args.accumulate(args.integers))
```

Assuming the Python code above is saved into a file called prog.py, it can be run at the command line and provides useful help messages:

```bash
$ python prog.py -h
usage: prog.py [-h] [--sum] N [N ...]
```

When run with the appropriate arguments, it prints either the sum or the max of the command-line integers:

```bash
$ python prog.py 1 2 3 4
4

$ python prog.py 1 2 3 4 --sum
10
```

<a href="https://www.linkedin.com/in/fredamouzgar/"><p style="font-size:8px">&#169; F.A, 2020</p></a>